In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
columns = ['Area', 'Perimeter', 'Compactness', 'KernelLength', 'KernelWidth',
           'AsymmetryCoeff', 'KernelGrooveLength', 'Class']

In [3]:
df = pd.read_csv('seeds_dataset.txt', sep='\s+', names=columns)


In [4]:
df

,Area,Perimeter,Compactness,KernelLength,KernelWidth,AsymmetryCoeff,KernelGrooveLength,Class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [5]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values - 1

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
X = np.expand_dims(X, axis=-1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

class CustomCNN(Model):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = Conv1D(16, kernel_size=3, activation='relu')
        self.pool = MaxPooling1D(pool_size=2)
        self.flatten = Flatten()
        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(3, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)


In [11]:
model = CustomCNN()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=1, callbacks=[early_stopping])

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5034 - loss: 1.0805 - val_accuracy: 0.8824 - val_loss: 0.9712
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8000 - loss: 0.9482 - val_accuracy: 1.0000 - val_loss: 0.8572
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8490 - loss: 0.8325 - val_accuracy: 0.8824 - val_loss: 0.7439
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8824 - loss: 0.7223 - val_accuracy: 0.8824 - val_loss: 0.6252
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8037 - loss: 0.6948 - val_accuracy: 1.0000 - val_loss: 0.5243
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8298 - loss: 0.5864 - val_accuracy: 1.0000 - val_loss: 0.4342
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8498 - loss: 0.5285 - val_accuracy: 1.0000 - val_loss: 0.3560
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8449 - loss: 0.4894 - val_accuracy: 1.0000 - val_loss:

In [13]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Sequential 모델 정확도: {acc*100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8790 - loss: 0.2798
Sequential 모델 정확도: 88.10%
